<a href="https://colab.research.google.com/github/lizhieffe/llm_knowledge/blob/main/examples/gdrive/Access_Google_Drive_via_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This colab demonstrates how to programming access Google Drive via API **without manual step**.

The core flow is to create a **Service Account** from GCP, and act as the **Service Account** to access GDrive to avoid manual step.

> **Important**: this doesn't work for **creating/updating file** in GDrive due to [a recent change](https://stackoverflow.com/questions/79736912/google-drive-api-service-accounts-do-not-have-storage-quota-leverage-shared-dri). The solution will be
  1. use Google Workspace account + Shared Drives, which costs money to register a Workspace account
  2. or OAuth delegation which I am not aware how to do.

In [17]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive

# Step 1 - Get GCP Service Account credential

> **Important**: this step assumes a GCP **Service Account** is already acreated.

## Option 1 - read user secret

This requires to store the credential in Colab's user secrect which is a one-off step.

In [18]:
from google.colab import userdata
data_stroage_secrect = userdata.get('gdrive_data_storage_service_account')

data_storage_secret_file = "/content/data_storage_secret.json"

with open(data_storage_secret_file, 'w') as f:
  f.write(data_stroage_secrect)

## Option 2 - directly user cred file

In Colab, this requires manually update the cred file to colab everytime.

In [19]:
%%script true

from google.colab import files
uploaded = files.upload()

In [20]:
%%script true

import json
original_cred_json_path = "/content/general-469917-4c9226e3c53d.json"

creds = service_account.Credentials.from_service_account_file(original_cred_json_path)
creds = creds.with_scopes(['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/drive'])

# Step 2 - Operate the GDrive

In [21]:
# @title Libs
def login_with_service_account():
  """
  Google Drive service with a service account.
  note: for the service account to work, you need to share the folder or
  files with the service account email.

  :return: google auth
  """
  # Define the settings dict to use a service account
  # We also can use all options available for the settings dict like
  # oauth_scope,save_credentials,etc.
  settings = {
              "client_config_backend": "service",
              "service_config": {
                  "client_json_file_path": data_storage_secret_file,
              }
          }
  # Create instance of GoogleAuth
  gauth = GoogleAuth(settings=settings)
  # Authenticate
  gauth.ServiceAuth()
  return gauth

gauth = login_with_service_account()
drive = GoogleDrive(gauth)

In [ ]:
# @title Example List files

# File_ID represents the dir or the file you want to operate.
#
# It is the last part of the URL string.
# For example:
# https://drive.google.com/drive/u/1/folders/1AT_CuwwJXWayI6W8PmkiLDgzq7YYYmwt
#
# Note: the file/dir needs to be shared to the service account.
#
file_id = '1AT_CuwwJXWayI6W8PmkiLDgzq7YYYmwt'

file_list = drive.ListFile({'q': f"'{file_id}' in parents and trashed=false"}).GetList()
for file in file_list:
    print('Title: %s, ID: %s' % (file['title'], file['id']))

In [22]:
# @title Example - Create/Update file

file4 = drive.CreateFile({'title':'test.json', 'mimeType':'application/json'})
file4.SetContentString('{"firstname": "John", "lastname": "Smith"}')
file4.Upload() # Upload file.
file4.SetContentString('{"firstname": "Claudio", "lastname": "Afshar"}')
file4.Upload() # Update content of the file.

print(f"File name = {file4['title']}, file id = {file4['id']}")

ApiRequestError: <HttpError 403 when requesting https://www.googleapis.com/upload/drive/v2/files?supportsAllDrives=true&alt=json&uploadType=resumable returned "Service Accounts do not have storage quota. Leverage shared drives (https://developers.google.com/workspace/drive/api/guides/about-shareddrives), or use OAuth delegation (http://support.google.com/a/answer/7281227) instead.". Details: "[{'message': 'Service Accounts do not have storage quota. Leverage shared drives (https://developers.google.com/workspace/drive/api/guides/about-shareddrives), or use OAuth delegation (http://support.google.com/a/answer/7281227) instead.', 'domain': 'usageLimits', 'reason': 'quotaExceeded'}]">